In [41]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

In [42]:
filepath = "pdf_files/southwindsor_park.pdf"

In [43]:
import os
os.environ["PATH"] = os.getenv('PATH') + ':' + '/opt/homebrew/Cellar/tesseract/5.3.4_1/bin' + ':' + '/opt/homebrew/Cellar/poppler/24.04.0/bin'
print(os.getenv('PATH'))

/Library/Developer/CommandLineTools/usr/bin:/Users/dileepbarla/Library/Python/3.9/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin/opt/homebrew/Cellar/poppler/24.04.0/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin:/opt/homebrew/Cellar/poppler/24.04.0/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin:/opt/homebrew/Cellar/poppler/24.04.0/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin:/opt/homebrew/Cellar/p

In [44]:
# Get elements
raw_pdf_elements = partition_pdf(
    filename=filepath,
    
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    # image_output_dir_path="./pdf_outputfiles/",
)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


In [45]:
print(raw_pdf_elements)

[<unstructured.documents.elements.CompositeElement object at 0x3645ca910>, <unstructured.documents.elements.CompositeElement object at 0x171513e50>, <unstructured.documents.elements.Table object at 0x35ee0d9d0>, <unstructured.documents.elements.Table object at 0x3645ca880>, <unstructured.documents.elements.Table object at 0x36206a040>, <unstructured.documents.elements.Table object at 0x36207a5e0>, <unstructured.documents.elements.CompositeElement object at 0x362046fa0>, <unstructured.documents.elements.CompositeElement object at 0x332330be0>, <unstructured.documents.elements.CompositeElement object at 0x362046cd0>, <unstructured.documents.elements.CompositeElement object at 0x362050be0>, <unstructured.documents.elements.CompositeElement object at 0x332330f70>, <unstructured.documents.elements.CompositeElement object at 0x36ec490d0>, <unstructured.documents.elements.CompositeElement object at 0x364d1c9d0>, <unstructured.documents.elements.CompositeElement object at 0x364d1c8b0>, <unstru

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage
import os
from dotenv import load_dotenv
from PIL import Image
import base64
import io

load_dotenv()

True

In [47]:
llm = ChatOpenAI(model="gpt-4-vision-preview")

In [48]:
# Function to convert image to base64
def image_to_base64(image_path):
    with Image.open(image_path) as image:
        buffered = io.BytesIO()
        image.save(buffered, format=image.format)
        img_str = base64.b64encode(buffered.getvalue())
        return img_str.decode('utf-8')

image_str = image_to_base64("figures/figure-1-1.jpg")

In [49]:
chat = ChatOpenAI(model="gpt-4-vision-preview",
                  max_tokens=1024)

msg = chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text" : "Whats the difference between residents and non-residents"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_str}"
                    },
                },
            ]
        )
    ]
)

In [51]:
print(msg.content)

In the context of the image, which appears to be a brochure or flyer for the Town of South Windsor Parks and Recreation Department, "residents" typically refers to individuals who live within the boundaries of South Windsor and are often eligible for certain benefits, discounts, or priority registration for local services, events, or programs. The flyer indicates there is a specific registration date for residents, which is likely earlier than for non-residents to allow them the first opportunity to sign up for activities.

"Non-residents," on the other hand, are individuals who do not live within the town or city limits of South Windsor. They may still be allowed to participate in programs and events but might have to pay higher fees, register on a different date, or have secondary priority compared to residents. In this case, the non-resident registration opens a week later than the resident registration, as per the dates listed on the flyer.

These distinctions are common in municip

In [52]:
chat = ChatOpenAI(model="gpt-4-vision-preview",
                  max_tokens=1024)

msg = chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text" : "Please give a summary of the image provided. Be descriptive."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_str}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)

The image is a promotional poster for the "Town of South Windsor Parks and Rec" for the Fall & Winter 2023/2024 season. At the top of the poster, the text is set against a vibrant red background which transitions to a green one as it mentions "PARKS AND REC". Below this section is a photograph of six young children standing behind a large, multicolored structure built from what appears to be large interlocking plastic bricks. The kids are smiling and seem to be enjoying themselves.

The children are wearing casual clothing, with some in t-shirts that feature familiar characters or logos. Below the photograph, there's a section with a green and white logo for the South Windsor Parks and Recreation and a list titled "NEW & UPCOMING" which highlights important dates for registration and mentions new preschool classes and the return of a "Haunted Torch Walk".

The overall design of the poster is colorful and appears to be aimed at a family audience, emphasizing community recreational activ